### Installing requirements and importing the package

In [2]:
# GraphToRDB requirements
# !pip3 install schema
# !pip3 install pyyaml

import GraphToRDB as g2r
import yaml

### Exploring the built-in schema validation

`g2r.validation` returns the `Schemas` object which contains `Schema` objects from the `schema` library.
These `Schema` objects have a `self.validate()` method which takes some data structure as an argument and compares it to the schema shape.

`g2r.validation` has the following `Schema` objects: `data_node`, `data_edge`, and `map`. 

When calling `Schema.validate()`, if the validation fails, a exception will be raised. When it passes, the method will return the input data.

**Example Validation (Pass)**

In [3]:
with open('example_map.yml') as file:
    config_map = yaml.safe_load(file)

g2r.validation.map.validate(config_map)

{'entity_tables': [{'map_node_label': ['Movie'],
   'table_name': 'e_movie',
   'columns': [{'map_node_property': 'tagline',
     'name': 'tagline',
     'dtype': 'str'},
    {'map_node_property': 'title', 'name': 'title', 'dtype': 'string'},
    {'map_node_property': 'released',
     'name': 'release_year',
     'dtype': 'int'}]},
  {'map_node_label': ['Person'],
   'table_name': 'e_person',
   'columns': [{'map_node_property': 'born',
     'name': 'year_of_birth',
     'dtype': 'int'},
    {'map_node_property': 'name', 'name': 'name', 'dtype': 'string'}]}],
 'relationship_tables': [{'map_edge_label': 'ACTED_IN',
   'table_name': 'r_acted_in',
   'map_from_node_label': ['Person'],
   'map_to_node_label': ['Movie'],
   'columns': [{'map_edge_property': 'roles',
     'name': 'roles',
     'dtype': 'list'}]},
  {'map_edge_label': 'DIRECTED',
   'table_name': 'r_directed',
   'map_from_node_label': ['Person'],
   'map_to_node_label': ['Movie']},
  {'map_edge_label': 'PRODUCED',
   'table_

**Example Validation (Fail)**

In [4]:
# Inserting a new key that violates the schema
config_map['new_key'] = None

try:
    g2r.validation.map.validate(config_map)
except Exception as error:
    message = str(error)

print(message)

# remove error
config_map.pop('new_key')

'Map Config (graph to relational)' Wrong key 'new_key' in {'entity_tables': [{'map_node_label': ['Movie'], 'table_name': 'e_movie', 'columns': [{'map_node_property': 'tagline', 'name': 'tagline', 'dtype': 'str'}, {'map_node_property': 'title', 'name': 'title', 'dtype': 'string'}, {'map_node_property': 'released', 'name': 'release_year', 'dtype': 'int'}]}, {'map_node_label': ['Person'], 'table_name': 'e_person', 'columns': [{'map_node_property': 'born', 'name': 'year_of_birth', 'dtype': 'int'}, {'map_node_property': 'name', 'name': 'name', 'dtype': 'string'}]}], 'relationship_tables': [{'map_edge_label': 'ACTED_IN', 'table_name': 'r_acted_in', 'map_from_node_label': ['Person'], 'map_to_node_label': ['Movie'], 'columns': [{'map_edge_property': 'roles', 'name': 'roles', 'dtype': 'list'}]}, {'map_edge_label': 'DIRECTED', 'table_name': 'r_directed', 'map_from_node_label': ['Person'], 'map_to_node_label': ['Movie']}, {'map_edge_label': 'PRODUCED', 'table_name': 'r_produced', 'map_from_node_l

## Using the Transformer Object

In [5]:
print(g2r.Transformer.__doc__)


    Takes
    * data: path to your valid JSON lines file
    * mapping: path to your mapping config YAML or an in-memory dict

    Validates the schema of the data and mappings on construction, will raise error during failure.
    If successful, access the output through the "rdb" attribute.
    You may write these sets to csvs or as SQL CREATE & INSERT statements.

    Attributes
    * data: the loaded and validated JSON lines file (list of dicts)
    * mapping: the loaded config (dict)
    * rdb: a dict containing the relational tables, initialized as empty dict at construction

    Output options (after calling generate_rdb_tables() method)
    * in-memory: access a dict of tables (tuple of tuples) from self.rdb
    * csv: call write_to_csv() method
    * sql statements: call generate_sql() method

    


In [6]:
tformer = g2r.Transformer(data='example_data.json', mapping='example_map.yml')